In [24]:
import pandas as pd
import numpy as np
import nltk
import string
import os
import torch
from collections import Counter
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from gensim.models import KeyedVectors

Carrega a BERT

In [26]:
#bert_tokenizer = AutoTokenizer.from_pretrained('/root/.cache/torch/transformers/neuralmind-bert-large-portuguese-cased/')
#bert_model = AutoModel.from_pretrained('/root/.cache/torch/transformers/neuralmind-bert-large-portuguese-cased/')

bertTokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
bertModel = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')

Carrega os dados de entrada

In [3]:
#pwd = %pwd
pwd = os.path.join(os.getcwd(), 'data')
print(pwd)

data = pd.read_csv(os.path.join(pwd, 'input/data_full.csv'), sep=';').replace({np.NaN: None})

D:\Mestrado\2020-02\Deep Learning\relation-extraction-deep-learning\data


In [4]:
data['class'] = data['Relacao'].apply(lambda x: 0 if x is None else 1)

In [5]:
data.head()

,Sentença,Entidade 1,Tipo Entidade 1,Relacao,Entidade 2,Tipo Entidade 2,class
0,A Kroton anunciou uma parceria com o Cubo Itaú...,Kroton,ORG,anunciou uma parceria com,Cubo Itaú,ORG,1
1,O evento Summit AgriHub reuniu nesta quinta-fe...,Summit AgriHub,ORG,reuniu em,Cuiabá,PLC,1
2,O evento Summit AgriHub reuniu nesta quinta-fe...,Summit AgriHub,ORG,None,Mato Grosso,PLC,0
3,O evento Summit AgriHub reuniu nesta quinta-fe...,Summit AgriHub,ORG,None,Brasil,PLC,0
4,"Ousuário consegue comparar prazos , condições ...",Bradesco,ORG,None,Banco do Brasil,ORG,0


Essa sera nossa estrategia de encoding para propagar esses dados pela rede neural

Verifica em quantos registros existem relacoes

In [6]:
Counter(data['class'])

Counter({1: 463, 0: 446})

Carrega o Word2Vec

In [7]:
w2vModel = KeyedVectors.load_word2vec_format(os.path.join(pwd, 'word embedding\cbow_s100.txt'))

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')
punct = string.punctuation

In [27]:
def filter_stopwords(tokens):
    return [i for i in tokens if i.lower() not in stopwords]

In [28]:
def tokenize_sentence_wv(sentence):
    sentence = nltk.word_tokenize(sentence)
    sentence = [i if i in w2vModel else 'unk' for i in sentence]
    return sentence

In [29]:
def to_wv_embeddings(sentence, word_embeddings):
    sentence = [i if i in word_embeddings else 'unk' for i in sentence]
    sentence = word_embeddings[sentence]
    return sentence

Etapa usada para converter o corpus em word embeddings simples (Word2Vec, GloVe, FastText, Wang2Vec, etc...)

In [30]:
corpus = data.apply(lambda x: (
    #filter_stopwords(tokenize_sentence_wv(x['Sentença'])),
    #to_wv_embeddings(filter_stopwords(tokenize_sentence_wv(x['Sentença'])), model),
    #to_wv_embeddings(filter_stopwords(tokenize_sentence_wv(x['Entidade 1'])), model),
    #to_wv_embeddings(filter_stopwords(tokenize_sentence_wv(x['Entidade 2'])), model)
    
    to_wv_embeddings(tokenize_sentence_wv(x['Sentença']), w2vModel),
    to_wv_embeddings(tokenize_sentence_wv(x['Entidade 1']), w2vModel),
    to_wv_embeddings(tokenize_sentence_wv(x['Entidade 2']), w2vModel)
    
), axis=1)

In [13]:
data.head(1)

,Sentença,Entidade 1,Tipo Entidade 1,Relacao,Entidade 2,Tipo Entidade 2,class
0,A Kroton anunciou uma parceria com o Cubo Itaú...,Kroton,ORG,anunciou uma parceria com,Cubo Itaú,ORG,1


In [14]:
len(data)

909

In [15]:
corpus.shape

(909,)

In [52]:
corpus[0]

(array([[-1.2163  ,  2.6971  , -3.4598  , ..., -3.2042  , -1.9964  ,
         -1.1786  ],
        [-1.2163  ,  2.6971  , -3.4598  , ..., -3.2042  , -1.9964  ,
         -1.1786  ],
        [ 1.6708  , -2.1491  ,  0.13539 , ..., -5.1199  ,  1.0708  ,
         -2.6411  ],
        ...,
        [-1.2163  ,  2.6971  , -3.4598  , ..., -3.2042  , -1.9964  ,
         -1.1786  ],
        [-0.01246 ,  2.9178  , -1.5546  , ..., -6.689   ,  1.238   ,
         -0.81424 ],
        [ 3.1289  ,  0.81339 ,  0.014809, ...,  3.3259  ,  0.15325 ,
         -3.7161  ]], dtype=float32),
 array([[-1.2163  ,  2.6971  , -3.4598  ,  1.5323  ,  0.76129 , -3.6975  ,
         -1.7161  ,  6.6654  , -0.99986 ,  6.7495  , -0.026322, -1.137   ,
          0.62763 , -3.593   ,  4.2524  , -0.99678 ,  1.9023  ,  2.5788  ,
          0.55511 ,  2.1383  ,  4.5285  ,  0.34367 , -0.81703 , -3.3503  ,
         -0.55212 ,  0.55477 ,  3.7589  , -4.8117  ,  1.4201  , -1.8194  ,
         -2.2001  ,  6.9085  ,  2.0602  , -4.353   , -2

Convertemos entao as possiveis entradas da rede neural para o formato correto utilizando a BERT <br>
Para facilitar o entendimento, vamos usar uma lista de tuplas, onde cada registro contem tres informacoes: a sentenca, a entidade 1 e a entidade 2 <br>

In [54]:
sample_txt = 'A Kroton anunciou uma parceria com o Cubo Itaú , que dará origem à vertical “Cubo Education” , um espaço dedicado ao fomento de edtechs – tecnologia aplicada à educação .'
tokens = bertTokenizer.tokenize(sample_txt)
token_ids = bertTokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

we = bertTokenizer.encode_plus(
  sample_txt,
  max_length=100,
  add_special_tokens=False, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
)
print(f'WE: {we}')

 Sentence: A Kroton anunciou uma parceria com o Cubo Itaú , que dará origem à vertical “Cubo Education” , um espaço dedicado ao fomento de edtechs – tecnologia aplicada à educação .
   Tokens: ['a', 'k', '##ro', '##ton', 'anunciou', 'uma', 'parceria', 'com', 'o', 'cub', '##o', 'it', '##au', ',', 'que', 'dar', '##a', 'origem', 'a', 'vertical', '“', 'cub', '##o', 'educa', '##tion', '”', ',', 'um', 'espa', '##co', 'dedicado', 'ao', 'fome', '##nto', 'de', 'ed', '##tec', '##h', '##s', '–', 'tecnologia', 'aplicada', 'a', 'educa', '##ca', '##o', '.']
Token IDs: [123, 1396, 157, 897, 3127, 230, 4495, 170, 146, 13435, 22280, 21627, 1382, 117, 179, 2822, 22278, 2008, 123, 14357, 1112, 13435, 22280, 6974, 1131, 22354, 117, 222, 1632, 303, 8055, 320, 11062, 234, 125, 902, 8948, 22296, 22281, 1379, 4277, 11107, 123, 6974, 304, 22280, 119]
WE: {'input_ids': [123, 1396, 157, 897, 3127, 230, 4495, 170, 146, 13435, 22280, 21627, 1382, 117, 179, 2822, 22278, 2008, 123, 14357, 1112, 13435, 22280, 6974, 1

In [65]:
we = bertTokenizer.encode(sample_txt)
bertModel.embeddings(2)

AttributeError: 'int' object has no attribute 'size'

In [57]:
corpus = data.head(10).apply(lambda x: (
    x['Sentença'], 
    to_bert_embeddings(x['Sentença'], bertModel),
    to_bert_embeddings(x['Entidade 1'], bertModel),
    to_bert_embeddings(x['Entidade 2'], bertModel)
), axis=1)

A
 
K
r
o
t
o
n
 
a
n
u
n
c
i
o
u
 
u
m
a
 
p
a
r
c
e
r
i
a
 
c
o
m
 
o
 
C
u
b
o
 
I
t
a
ú
 
,
 
q
u
e
 
d
a
r
á
 
o
r
i
g
e
m
 
à
 
v
e
r
t
i
c
a
l
 
“
C
u
b
o
 
E
d
u
c
a
t
i
o
n
”
 
,
 
u
m
 
e
s
p
a
ç
o
 
d
e
d
i
c
a
d
o
 
a
o
 
f
o
m
e
n
t
o
 
d
e
 
e
d
t
e
c
h
s
 
–
 
t
e
c
n
o
l
o
g
i
a
 
a
p
l
i
c
a
d
a
 
à
 
e
d
u
c
a
ç
ã
o
 
.
K
r
o
t
o
n
C
u
b
o
 
I
t
a
ú
O
 
e
v
e
n
t
o
 
S
u
m
m
i
t
 
A
g
r
i
H
u
b
 
r
e
u
n
i
u
 
n
e
s
t
a
 
q
u
i
n
t
a
-
f
e
i
r
a
 
(
1
9
/
0
4
)
 
,
 
e
m
 
C
u
i
a
b
á
 
,
 
p
r
o
d
u
t
o
r
e
s
 
r
u
r
a
i
s
 
v
i
n
d
o
s
 
d
e
 
v
á
r
i
o
s
 
m
u
n
i
c
í
p
i
o
s
 
d
e
 
M
a
t
o
 
G
r
o
s
s
o
 
,
 
p
e
s
q
u
i
s
a
d
o
r
e
s
 
e
 
e
m
p
r
e
s
a
s
 
d
e
 
t
e
c
n
o
l
o
g
i
a
s
 
d
o
 
B
r
a
s
i
l
 
p
a
r
a
 
d
e
b
a
t
e
r
 
a
s
 
n
o
v
i
d
a
d
e
s
 
i
n
c
o
r
p
o
r
a
d
a
s
 
a
o
s
 
s
i
s
t
e
m
a
s
 
d
e
 
p
r
o
d
u
ç
ã
o
 
.
S
u
m
m
i
t
 
A
g
r
i
H
u
b
C
u
i
a
b
á
O
 
e
v
e
n
t
o
 
S
u
m
m
i
t
 
A
g
r
i
H
u
b
 
r
e
u
n
i
u
 
n
e
s
t
a
 
q
u
i
n
t
a
-
f


## Elaboracao da rede neural

Podemos fazer com que a rede tenha apenas um ou mais modelos internos. Por exemplo, um modelo para verificar se existe relacao e outro para extrair essa relacao

É possivel receber modelos por parametros dentro das classes de nn.Module. Assim, incorporamos o modelo recebido as configuracao da rede que estamos propondo. <br>
Abaixo é possivel ver que incorporamos a BERT nos modelos ModeloVerificaRelacao e ModeloExtraiRelacao. Alem disso, incorporamos esses dois modelos tambem em ModeloDaniel

Deste modo, temos as seguintes possibilidades:<br>
    - Os modelos ModeloVerificaRelacao e ModeloExtraiRelacao podem propagar suas entradas pela rede da BERT<br>
    - Treinar apenas o ModeloDaniel e fazer com que ele propague o erro para ModeloVerificaRelacao, ModeloExtraiRelacao e BERT (Isso é considerado um fine-tuning da BERT; O desafio sera propagar as saidas de ModeloVerificaRelacao para ModeloExtraiRelacao)<br>
    - Treinar ModeloVerificaRelacao e ModeloExtraiRelacao separados e depois adicionalos ao ModeloDaniel apenas para unificacao (Essa abordagem é mais didatica; O desafio sera elaborar um metodo de treinamento consistente que permita apenas um treinamento, caso contrario deverao existir varios em separado e apenas um local para unificar suas chamadas)<br>

In [ ]:
import torch.nn as nn

In [ ]:
class ModeloVerificaRelacao(nn.Module):
    def __init__(self, embedding_model, tokenizer):
        super().__init__()
        self.embedding_model = embedding_model
        self.tokenizer = tokenizer
        
        self.drop_prob = 0.0
        self.n_layers = 1
        self.n_hidden = 1
        self.lr = 0.1
        tokens = []
        
        # creating character dictionaries
        self._fields = tokens
        self._int2field = dict(enumerate(self._fields))
        self._field2int = {ch: ii for ii, ch in self._int2field.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self._fields), self.n_hidden, self.n_layers, 
                            dropout=self.drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(self.drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(self.n_hidden, len(self._fields))
        
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out
        
class ModeloExtraiRelacao(nn.Module):
    def __init__(self, embedding_model, tokenizer):
        super().__init__()
        self.embedding_model = embedding_model
        self.tokenizer = tokenizer
        
        self.drop_prob = 0.0
        self.n_layers = 1
        self.n_hidden = 1
        self.lr = 0.1
        tokens = []
        
        # creating character dictionaries
        self._fields = tokens
        self._int2field = dict(enumerate(self._fields))
        self._field2int = {ch: ii for ii, ch in self._int2field.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self._fields), self.n_hidden, self.n_layers, 
                            dropout=self.drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(self.drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(self.n_hidden, len(self._fields))
        
    def forward(self, inputs, relation):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(inputs, relation)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
        
class ModeloDaniel(nn.Module):
    def __init__(self, verifica, extrai):
        super().__init__()
        self._modelo1 = verifica
        self._modelo2 = extrai
        
    def forward(self, inputs):
        out = self._modelo1(inputs)
        out = self._modelo2(inputs, out)
        
        return out


In [ ]:
net = ModeloDaniel(
    verifica = ModeloVerificaRelacao(bert_model, bert_tokenizer),
    extrai = ModeloExtraiRelacao(bert_model, bert_tokenizer)
)
print(net)

Os modelos podem ser adicionados e treinarem junto com a rede geral, ou entao apenas adicionados postariormente

## Classificacao com MLPClassifier

In [ ]:
corpus[0][0]

Verificando o que ha dentro da variavel corpus

In [ ]:
corpus[0][0]

A primeira posicao contem os embeddings da sentenca

O formato é (batch_size, elementos, embeddings)

In [ ]:
corpus[0][1].shape

A segunda e terceira posicoes contem os dados da Entidade 1

In [ ]:
print(corpus[0][2].shape)
print(corpus[0][3].shape)

Entao com isso temos tensores de 1, por N por 1024

Para entrar no algoritmo, o ideal é que ocorra um flattern desses dados, ou seja, transformar cada uma dessas matrizes em vetores

In [ ]:
corpus = [
    (
        corpus[0][1].squeeze().sum(axis=0),
        corpus[0][2].squeeze().sum(axis=0),
        corpus[0][3].squeeze().sum(axis=0),
    )
    for x in corpus
]

In [ ]:
print(corpus[0][0].shape)
print(corpus[0][1].shape)
print(corpus[0][2].shape)

O modelo nao pode ter tres vetores de entrada, entao temos de concatenar os vetores

In [ ]:
torch.cat((corpus[0][0],corpus[0][1],corpus[0][2]), dim=0).shape

In [ ]:
corpus = [torch.cat((x[0],x[1],x[2]), dim=0) for x in corpus]

In [ ]:
corpus[0].shape

In [ ]:
from sklearn.neural_network import MLPClassifier
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics

In [ ]:
def evaluate_binary_classification(y_true, y_pred, pos_label=1):
    
    def metric_auc(y_true, y_pred, pos_label=True):
        try:
            fpr, tpr, _threshold = metrics.roc_curve(
                y_true,
                y_pred,
                pos_label=pos_label
            )
            return metrics.auc(fpr, tpr)
        except:
            return None

    dct = {
        'ACCURACY': metrics.accuracy_score(y_true, y_pred),
        'F1': metrics.f1_score(y_true, y_pred, average='weighted', pos_label=pos_label),
        'F1_MICRO': metrics.f1_score(y_true, y_pred, average='micro', pos_label=pos_label),
        'F1_MACRO': metrics.f1_score(y_true, y_pred, average='macro', pos_label=pos_label),
        'PRECISION': metrics.precision_score(y_true, y_pred, average='binary', pos_label=pos_label),
        'RECALL': metrics.recall_score(y_true, y_pred, average='binary', pos_label=pos_label),
        'AUC': metric_auc(y_true, y_pred),
        'KAPPA': metrics.cohen_kappa_score(y_true, y_pred)
    }
    return {k: 0.0 if not v else v for k, v in dct.items()}

In [ ]:
model =  MLPClassifier(
    hidden_layer_sizes=(100,),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    power_t=0.5,
    max_iter=200,
    shuffle=True,
    random_state=None,
    tol=0.0001,
    verbose=False,
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    n_iter_no_change=10,
    max_fun=15000
)

Para usar como entrada em uma MLP simples, temos de converter de lista de tensores para lista de listas

In [ ]:
corpus = [x.detach().numpy() for x in corpus]

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(corpus, data['class'], test_size=0.2, random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = [model.predict(x.reshape(1,-1)).tolist()[0] for x in X_test]

In [ ]:
print(f'y_true: {y_test.tolist()}')
print(f'y_pred: {y_pred}')

In [ ]:
evaluate_binary_classification(y_test, y_pred)